<a href="https://colab.research.google.com/github/weathon/kNN_Research/blob/main/Credit_Card_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cp kaggle.json /root/.kaggle/
!kaggle datasets download -d sakshigoyal7/credit-card-customers
!unzip credit-card-customers.zip

credit-card-customers.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  credit-card-customers.zip
  inflating: BankChurners.csv        


In [3]:
import pandas as pd
import numpy as np

### Data Washing

In [4]:
BankChurners = pd.read_csv("BankChurners.csv")

In [5]:
del BankChurners["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1"]
del BankChurners["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"]

In [6]:
Y = BankChurners["Attrition_Flag"]
del BankChurners["Attrition_Flag"]
del BankChurners["CLIENTNUM"]
X = BankChurners

In [7]:
Y = Y.replace("Existing Customer",0).replace("Attrited Customer",1)

In [8]:
X["Gender"] = X["Gender"].replace("M",1).replace("F",0)
X["Marital_Status"] = X["Marital_Status"].replace("Married",0).replace("Single",1).replace("Divorced",2).replace("Unknown",3)
X["Education_Level"] = X["Education_Level"].replace("Uneducated",0).replace("High School",1).replace("College",2).replace("Graduate",3).replace("Post-Graduate",4).replace("Doctorate",5).replace("Unknown",-1)
X["Card_Category"] = X["Card_Category"].replace("Blue",0).replace("Silver",1).replace("Gold",2).replace("Platinum",3)
X["Income_Category"] = X["Income_Category"].replace("Less than $40K",0).replace("$40K - $60K",1).replace("$60K - $80K",2).replace("$80K - $120K",3).replace("$120K +",4).replace("Unknown",-1)

### Baseline: tradional kNN

In [24]:
def result(sum0, sum1):
  return 1 if sum1>sum0 else 0

In [79]:
X = np.array(X)
Y = np.array(Y)
indexs = np.arange(len(X))

np.random.shuffle(indexs)
X = X[indexs]
Y = Y[indexs]
def train_step(A):
  global X,Y
  r = int(len(X)*0.9)
  npX = np.array(X[:r])
  npY = np.array(Y[:r])
  X_val = np.array(X[r:])
  Y_val = np.array(Y[r:])

  error = 0
  TruePositives = 0
  FalseNegatives = 0
  FalsePositives = 0
  TrueNegatives = 0
  MSE = 0

  for sample in range(len(X)-r): # change this from 20 to r decreased the accuracy a alot
    distance = np.sum((npX - X_val[sample])**2,axis=1)
    sum0 = 0
    sum1 = 0
    for i in range(100):
      m = np.argmin(distance)
      distance[m] = 1000000000 
      sum0 += 1/(distance[m]+1) * (1-npY[m])
      sum1 += 1/(distance[m]+1) * (npY[m])

    if(Y_val[sample] != result(sum0, sum1)):
      error+=1
    
    if Y_val[sample] == result(sum0, sum1) == 1:
      TruePositives += 1

    if result(sum0, sum1) == 0 and Y_val[sample] == 1:
      FalseNegatives +=1

    if result(sum0, sum1) == 1 and Y_val[sample] == 0:
      FalsePositives +=1

    if Y_val[sample] == result(sum0, sum1) == 0:
      TrueNegatives += 1

    MSE += (sum1/(sum0+sum1) - Y_val[sample])**2

  # print(1-error/r)
  # print(TruePositives / (TruePositives + FalseNegatives))
  # print(TruePositives / (TruePositives + FalsePositives))
  # print(MSE/(len(X)-r)) 
  # return MSE/(len(X)-r)
  return TruePositives / (TruePositives + FalsePositives)

In [93]:
# Guess and try method for GD
#xiamianyangttnianbenlaishibuyiyangdewhyxianzaiyiyang
A = np.ones(19)
def ff(A):
  return np.sum(A**2)
for epoches in range(10):
  loss = ff(A)
  A[1] += 0.1
  loss2 = ff(A)
  slope = (loss2-loss)/0.1
  A[1]-=slope*10
  print(loss)

19.0
414.01000000000334
142297.84000000628
51375941.29005829
18546463191.3943
6695277863144.764
2416995224292102.0
8.725353451689786e+17
3.149852565874189e+20
1.1370499026228518e+23


In [84]:
slope

-1.8999999999999773

In [67]:
A[1]=1000000
A[2]=1000000
A[3]=1000000 #haishiyoubianhuadewhyzhiqinamieyo taychikunyun 1 bushi0xiamian

In [ ]:
y=[]
for x in range(100):
  loss=train_step(A)
  A[0]+=0.1
  y.append(loss)

kunshouzangbushoulie kun ttyunnashou huang xkou yuntt chaojikun xianhuachulai?xianbusuiji?kun

0.9859556725916173
0.37058823529411766
0.75
0.08788173741362298


0.08788173741362298

In [345]:
sum1/(sum0+sum1)

0.18000000000000022

In [335]:
sum

-9.999999989999978e-08

In [328]:
distance[8]

4043056.074194

### Baseline 2 (NN)

In [1]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds
import numpy as np
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [18]:

model = tf.keras.models.Sequential([
  # RBG, because pineapple is yellow
  tf.keras.layers.Input(X.shape[1]),
  tf.keras.layers.Dense(40, activation="relu"),
  tf.keras.layers.Dense(50, activation="relu"),
  tf.keras.layers.Dense(60, activation="relu"),
  tf.keras.layers.Dense(50, activation="relu"),
  tf.keras.layers.Dense(20, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid"),
])
model.summary()
# print(model.output_shape)
model.compile(optimizer=tf.keras.optimizers.Adam(),
          loss='binary_crossentropy',metrics=["accuracy"])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 40)                800       
                                                                 
 dense_17 (Dense)            (None, 50)                2050      
                                                                 
 dense_18 (Dense)            (None, 60)                3060      
                                                                 
 dense_19 (Dense)            (None, 50)                3050      
                                                                 
 dense_20 (Dense)            (None, 20)                1020      
                                                                 
 dense_21 (Dense)            (None, 10)                210       
                                                                 
 dense_22 (Dense)            (None, 1)                

In [21]:
model.fit(X,Y)

317/317 [==============================] - 1s 4ms/step - loss: 0.6061 - accuracy: 0.8216
